<a href="https://colab.research.google.com/github/TrollRider-Kristian/Springboard-AI-Mini-Projects/blob/main/codebasics_bagging_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Force Pandas to display ALL columns when I call the .head function
# https://stackoverflow.com/questions/11361985/output-data-from-all-columns-in-a-dataframe-in-pandas
pandas.set_option ('display.max_columns', None)
heart_disease_raw = pandas.read_csv ("heart-disease.csv")
print (heart_disease_raw.head(10))

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   
5   39   M           NAP        120          339          0     Normal    170   
6   45   F           ATA        130          237          0     Normal    170   
7   54   M           ATA        110          208          0     Normal    142   
8   37   M           ASY        140          207          0     Normal    130   
9   48   F           ATA        120          284          0     Normal    120   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1     

In [15]:
heart_disease_raw.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [16]:
# The Z-score, also called the Standard Score: https://en.wikipedia.org/wiki/Standard_score
# I was advised to remove outliers with a Z-score < -3 or > 3, as in any row with a value
# more than 3 standard deviations away from the mean.

def calculate_z_score (dataset, column_name):
  get_column = dataset [column_name]
  return (get_column - get_column.mean()) / get_column.std()

# Per the describe() call above, all the quantifiable columns with means and standard deviations
# for which a Z-score exists are: Age, RestingBP, Cholesterol, FastingBS, MaxHR, and Oldpeak.
# We omit the target 'HeartDisease' column.

heart_disease_raw ['Z-score-Age'] = calculate_z_score (heart_disease_raw, 'Age')
heart_disease_raw ['Z-score-RestingBP'] = calculate_z_score (heart_disease_raw, 'RestingBP')
heart_disease_raw ['Z-score-Cholesterol'] = calculate_z_score (heart_disease_raw, 'Cholesterol')
heart_disease_raw ['Z-score-FastingBS'] = calculate_z_score (heart_disease_raw, 'FastingBS')
heart_disease_raw ['Z-score-MaxHR'] = calculate_z_score (heart_disease_raw, 'MaxHR')
heart_disease_raw ['Z-score-Oldpeak'] = calculate_z_score (heart_disease_raw, 'Oldpeak')

print (heart_disease_raw.head(10))

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   
5   39   M           NAP        120          339          0     Normal    170   
6   45   F           ATA        130          237          0     Normal    170   
7   54   M           ATA        110          208          0     Normal    142   
8   37   M           ASY        140          207          0     Normal    130   
9   48   F           ATA        120          284          0     Normal    120   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  Z-score-Age  \
0              N      0.0       Up         